In [13]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [14]:
import os

from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'
import numpy as np

from pandeia.engine.perform_calculation import perform_calculation
from pandeia.engine.calc_utils import build_default_calc, build_default_source

import time

from package.helpers import test_lens
from package.plots import plot

In [15]:
oversample_factor_list = [1, 3]
execution_time = []

figure_dir = os.path.join(repo_path, 'figures', 'diagnostics', 'pixel_density_test')
array_dir = os.path.join(repo_path, 'output', 'arrays', 'diagnostics', 'pixel_density_test')

lens = test_lens.TestLens()

In [16]:
for oversample_factor in tqdm(oversample_factor_list):
    side = 5
    num_pix = 45 * oversample_factor

    image = lens.get_array(num_pix, side)

    plot.plot(filepath=os.path.join(figure_dir, f'model_{oversample_factor}.png'),
              title=f'Lenstronomy model\nOversampling factor {oversample_factor}', array=image)
    np.save(os.path.join(array_dir, f'model_{oversample_factor}'), image)

    # start timer
    start = time.time()

    calc = build_default_calc('roman', 'wfi', 'imaging')

    # scene size settings
    calc['configuration']['dynamic_scene'] = True
    calc['configuration']['max_scene_size'] = 5

    # change filter
    calc['configuration']['instrument']['filter'] = 'f106'

    # normalize (np.interp for now)
    max = np.max(image)
    min = np.min(image)
    image = np.interp(image, (min, max), (0.00001, 0.001))

    i = 0
    print('Converting to point sources...')
    for row_number, row in tqdm(enumerate(image)):
        for item_number, item in enumerate(row):
            if i != 0:
                calc['scene'].append(build_default_source(geometry="point"))
            calc['scene'][i]['position']['x_offset'] = (item_number * (1 / 9) * (
                        1 / oversample_factor)) + lens.ra_at_xy_0  # arcsec
            calc['scene'][i]['position']['y_offset'] = (row_number * (1 / 9) * (
                        1 / oversample_factor)) + lens.dec_at_xy_0  # arcsec
            calc['scene'][i]['spectrum']['normalization']['norm_flux'] = item

            i += 1
    print('Point source conversion complete')

    print('Performing Pandeia calculation...')
    results = perform_calculation(calc)
    print('Pandeia calculation complete')

    # stop timer
    stop = time.time()
    execution_time.append(stop - start)

    detector = results['2d']['detector']

    # TODO TEMP! flip image
    detector = np.flipud(detector)
    # detector = np.fliplr(detector)

    plt.imshow(detector)
    plt.title('Pandeia simulation\nOversampling factor ' + str(oversample_factor))
    plt.show()
    plt.savefig(os.path.join(figure_dir, f'pandeia_{oversample_factor}.png'))

    # save this numpy array
    np.save(os.path.join(array_dir, f'detector_{oversample_factor}'), detector)

  0%|          | 0/2 [00:00<?, ?it/s]


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
plt.scatter(oversample_factor_list, execution_time)
plt.show()